In [1]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

MAP_VERSION = 3
STATS_TYPES = ['Count','Percent','nshft', 'pshft','growth', 'pshftna','growthna','Labels']


In [2]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [3]:
STATE_FILES = [
    "tl_2010_01_bg10.json",
    "tl_2010_02_bg10.json",
    "tl_2010_04_bg10.json",
    "tl_2010_05_bg10.json",
    "tl_2010_06_bg10.json",
    "tl_2010_08_bg10.json",
    "tl_2010_09_bg10.json",
    "tl_2010_10_bg10.json",
    "tl_2010_11_bg10.json",
    "tl_2010_12_bg10.json",
    "tl_2010_13_bg10.json",
    "tl_2010_15_bg10.json",
    "tl_2010_16_bg10.json",
    "tl_2010_17_bg10.json",
    "tl_2010_18_bg10.json",
    "tl_2010_19_bg10.json",
    "tl_2010_20_bg10.json",
    "tl_2010_21_bg10.json",
    "tl_2010_22_bg10.json",
    "tl_2010_23_bg10.json",
    "tl_2010_24_bg10.json",
    "tl_2010_25_bg10.json",
    "tl_2010_26_bg10.json",
    "tl_2010_27_bg10.json",
    "tl_2010_28_bg10.json",
    "tl_2010_29_bg10.json",
    "tl_2010_30_bg10.json",
    "tl_2010_31_bg10.json",
    "tl_2010_32_bg10.json",
    "tl_2010_33_bg10.json",
    "tl_2010_34_bg10.json",
    "tl_2010_35_bg10.json",
    "tl_2010_36_bg10.json",
    "tl_2010_37_bg10.json",
    "tl_2010_38_bg10.json",
    "tl_2010_39_bg10.json",
    "tl_2010_40_bg10.json",
    "tl_2010_41_bg10.json",
    "tl_2010_42_bg10.json",
    "tl_2010_44_bg10.json",
    "tl_2010_45_bg10.json",
    "tl_2010_46_bg10.json",
    "tl_2010_47_bg10.json",
    "tl_2010_48_bg10.json",
    "tl_2010_49_bg10.json",
    "tl_2010_50_bg10.json",
    "tl_2010_51_bg10.json",
    "tl_2010_53_bg10.json",
    "tl_2010_54_bg10.json",
    "tl_2010_55_bg10.json",
    "tl_2010_56_bg10.json",
]

In [4]:
# Input Files
# 2 => "02"
def formatted_state_fips_code(fips_number):
    return f'{fips_number:02d}'

#  block_group_geojson_filename(state_fips_code=46, year=2010)
# tl_2010_46_bg10.json
def block_group_geojson_filename(state_fips_code=1, census_year=2010):
    year_string = str(census_year)
    fips_code_string = formatted_state_fips_code(state_fips_code)
    return f'tl_{year_string}_{fips_code_string}_bg10.json'

# wa_washington_zip_codes_geo.min.json
# def zip_code_geojson_filename(state_fips_code=1):
#     formatted_fips_code = formatted_state_fips_code(state_fips_code)
#     state_abbr = fips_codes[formatted_fips_code]["abbr"].lower()
#     state_full_name = fips_codes[formatted_fips_code]["full_name"]
#     return f'{state_abbr}_{state_full_name}_zip_codes_geo.min.json'


# def zip_geojson_files_for_market(market_data):
#     fips_codes = json.loads(market_data['StateFips'])
#     files = []
#     for fips_code in fips_codes:
#         files.append(zip_code_geojson_filename(fips_code))
#     return files

def bg_geojson_files_for_market(market_data, census_year):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(block_group_geojson_filename(state_fips_code=int(fips_code), census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

# def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
#     data_type = "zip"
#     csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
#     data_rows = []
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)
#     zip_codes = []
#     for data_row in data_rows:
#         zip_codes.append(data_row['Zip'].zfill(5))
#     print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
#     return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return block_groups
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "bg"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}_Mapping_{file_type}_Data.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}_Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}/data/{market_prefix}_{data_type}_data_v{MAP_VERSION}.json'

def published_map_settings_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_settings_data_v{MAP_VERSION}.json'

def published_map_labels_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_labels_v{MAP_VERSION}.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='bg'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, map_settings_data, bg_data_rows, stats_type):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        if block_group in features:
            found = True
            feature = features[block_group]
            new_feature = copy.deepcopy(feature)
            properties = new_feature['properties']
            properties.pop("ALAND10",0)
            properties.pop("AWATER10",0)
            properties.pop("BLKGRPCE10",0)
            properties.pop("COUNTYFP10",0)
            properties.pop("FUNCSTAT10",0)
            properties.pop("MTFCC10",0)
            properties.pop("NAMELSAD10",0)
            properties.pop("STATEFP10",0)
            properties.pop("TRACTCE10",0)
            new_feature['properties'] = properties
            for key in list(data_row.keys())[0:]:
                if key not in  ['BG','LAT','LONG'] and stats_type in key: #and 'Count' in key:
                    if data_row[key] and data_row[key] != 'inf':
                        translated_key = map_settings_data[key]['mapbox_segment']
                        value = ast.literal_eval(data_row[key].replace(',',''))
                        value = int(value * 1000000) / 1000000
                        new_feature['properties'][translated_key] =  value
            new_features_to_keep.append(new_feature)
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)


def create_data_json(features):
    data = []
    for feature in features:
        #centroid = {
        #    "GEOID10": feature["properties"]["GEOID10"],
        #    "INTPTLAT10": feature["properties"]["INTPTLAT10"],
        #    "INTPTLON10": feature["properties"]["INTPTLON10"],
        #}
        datum = feature["properties"]
        data.append(datum)
    return data

def write_geojson(geojson_data, market_prefix, data_type='bg', stats_type='Count',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{market_prefix}/data/{market_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json')
    
def write_data_json(json_data, market_prefix, data_type='bg', stats_type='count',published_data_base_path = "./public"):
    json_string = json.dumps(json_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{market_prefix}_{stats_type.lower()}_data_v{MAP_VERSION}.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote centroid json data to  {published_data_base_path}/{market_prefix}/data/{market_prefix}_{stats_type.lower()}_data_v{MAP_VERSION}.json')


def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "bg"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, market_file_prefix, base_path="./public", data_type = "bg"):
    processed_settings_data = OrderedDict()
    segment_number = 0
    for row in settings_data:
        segment_number += 1
        if row["Display_Data_Type"] == "HH_Count":
            row["Display_Data_Type"] = "HH Count"
        mapbox_segment_key = f's{segment_number}'
        row['mapbox_segment'] = mapbox_segment_key
        processed_settings_data[row['UniqueSeriesString']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(market_file_prefix,  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    return processed_settings_data
    

def create_bg_labels(market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(market_file_prefix,  data_type , published_data_base_path=published_data_base_path)
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "./public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            row['CenterLat'] = float(row['CenterLat'])
            row['CenterLong'] = float(row['CenterLong'])
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json')
    
    
def create_tileset_data_recipe_file(market_file_prefix, published_data_base_path, stats_type):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_{stats_type.lower()}_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
          'tiles': {
            'layer_size': 2500
          }
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
    

def create_tileset_labels_recipe_file(market_file_prefix, published_data_base_path):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{market_file_prefix}_labels_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{market_file_prefix}_labels_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
            
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{market_file_prefix}_labels_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
        
# def mstds_convert_file(market_file_prefix, published_data_base_path, filetype='data'):
#     recipe_file_path = os.path.join(published_data_base_path, market_file_prefix,'labels','mts-recipe.json' )
#     market_file_prefix, published_data_base_path
    

In [5]:
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
def create_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    nationwide_features = {}
    for file in STATE_FILES:
        print(f'processing: {file}')
        geosjon_path = os.path.join(geojson_files_base_path,file)
        features = read_geo_gson_file(geosjon_path)
        for feature in features:
            key = feature['properties']["GEOID10"]
            nationwide_features[key] = feature
    json_string = json.dumps(nationwide_features, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson data to  {geojson_files_base_path}/nationwide_bg.json')

def read_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    # Opening JSON file
    f = open(geosjon_path)
    # returns JSON object as 
    # a dictionary
    features = json.load(f)
    # Closing file
    f.close()
    return features

def write_nation_bg_centroids_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    features = read_nation_bg_features_json(geojson_files_base_path)
    centroids = []
    for key in features.keys:
        feature = features[key]
        centroid = {
            "GEOID10": key,
            "INTPTLAT10": feature["properties"]["INTPTLAT10"],
            "INTPTLON10": feature["properties"]["INTPTLON10"],
        }
        centroids.append(centroid)
    json_string = json.dumps(centroids, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg_centroids.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote bg centroids data to  {geojson_files_base_path}/nationwide_bg_centroids.json')
    
        
        
    

In [6]:
def create_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        # segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(region,year)
        print(urban, suburban, exurban)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'] = urban
        data_json[region]['data'][year]['SubUrban'] = suburban
        data_json[region]['data'][year]['ExUrban'] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json')



In [7]:
def create_comparison_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        display_region = row['DisplayRegion']
        year = row['Year']
        age_segment=row['Display_Age_Segment']
        income_segment=row['Display_Income_Segment']
        key = f'{region}-{year}-{age_segment}-{income_segment}'
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(key)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        isMetro = row['IsMetro']
        data_json[key] = {
            "year": year,
            "ageSegment": age_segment,
            "incomeSegment": income_segment,
            "urban": urban,
            "suburban": suburban,
            "exurban": exurban,
            "isMetro": isMetro,
        }
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json')
    

In [8]:
def create_data_options_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    year_options_set = set()
    age_segment_options_set = set()
    income_segment_options_set = set()
    region_options_list = []
    for row in chart_data_rows:
        year_options_set.add(row["Year"])
        age_segment_options_set.add(row["Display_Age_Segment"])
        income_segment_options_set.add(row["Display_Income_Segment"])
        region = {
            "region": row["Region"],
            "displayRegion": row["DisplayRegion"],
            "isMetro": row["IsMetro"],
        }
        region_options_list.append(region)
    year_options_list = list(year_options_set)
    year_options_list.sort()
    age_segment_options_list = list(age_segment_options_set)
    age_segment_options_list.sort()
    income_segment_options_list = list(income_segment_options_set)
    income_segment_options_list.sort()
    region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
    dataOptions = {
        "yearOptions": year_options_list,
        'ageSegmentOptions': age_segment_options_list,
        'incomeSegmentOptions': income_segment_options_list,
        "regionOptions": region_options
    }
    data_options_string = json.dumps(dataOptions, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/data_options_v{MAP_VERSION}.json', "w")
    f.write(data_options_string)
    f.close()
    print(f'Wrote data options data to  {published_data_base_path}/data_options_v{MAP_VERSION}.json')


In [9]:
def process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year):
    nationwide_bg_features = read_nation_bg_features_json(geojson_files_base_path)
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows:
        #         if market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
        #             #if True:
        #             #print(market_data['MarketName'])
        #if market_data['MarketName'] in ['Ohio','ClevelandOH']:
        if True: # market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
        
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            market_prefix = market_file_prefix

            for stats_type in STATS_TYPES:
                create_tileset_data_recipe_file(market_file_prefix, published_data_base_path,stats_type)
                create_tileset_labels_recipe_file(market_file_prefix, published_data_base_path)
            # Block Group Processing
            data_type = 'bg'
            map_settings_data = read_map_settings(market_data['MarketName'], base_path=source_data_base_path, data_type = "bg")
            map_settings = write_settings_data(map_settings_data, market_prefix, base_path=published_data_base_path, data_type = "bg")
            block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            for stats_type in STATS_TYPES:
                features_to_use = create_bg_features(nationwide_bg_features, map_settings, block_group_rows, stats_type)
                new_geojson = create_geojson(features_to_use)
                write_geojson(new_geojson, market_prefix, data_type, stats_type,published_data_base_path=published_data_base_path)
                if stats_type.lower() != 'labels':
                    new_data_json = create_data_json(features_to_use)
                    write_data_json(new_data_json, market_prefix.lower(),data_type, stats_type.lower(),published_data_base_path)
            create_bg_labels(market_file_prefix, base_path=source_data_base_path , data_type = "bg" , published_data_base_path=published_data_base_path)
            market_count += 1
    # Finished states - create overall settings file    
    create_geojson_settings(csv_file_path, published_data_base_path = "./public")
    print(f'*** COMPLETED {market_count} markets')
    print(completed_markets)
                          
        
# source_data_base_path = "./source_data"
# source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie/"
# published_data_base_path = "./public"
# geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# census_year = 2010

In [29]:
# Set up input and output locations
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# Source csv data path
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie"
# Output data path - all of the files in this folder should be uploaded to the cms GeoSpatialTool/v1 folder
published_data_base_path = "./public"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
# Census year for geojson
census_year = 2010


# Process the data
process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year)
# create_chart_json(published_data_base_path)
# create_comparison_chart_json(published_data_base_path)
create_data_options_json(published_data_base_path)


Market Data: 1
Loaded 374 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/AbileneTX_Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/abilenetx_bg_settings_data_v3.json
Loaded 144 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/abilenetx_Mapping_BG_Data.csv
Processed 144 features to use
Created GeoJson with  144 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/abilenetx/data/abilenetx_count_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/abilenetx/data/abilenetx_count_data_v3.json
Processed 144 features to use
Created GeoJson with  144 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/abilenetx/data/abilenetx_percent_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/abilenetx/data/abilenetx_percent_data_v3.json
Processed 144

Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/albanyor/data/albanyor_count_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/albanyor/data/albanyor_count_data_v3.json
Processed 85 features to use
Created GeoJson with  85 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/albanyor/data/albanyor_percent_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/albanyor/data/albanyor_percent_data_v3.json
Processed 85 features to use
Created GeoJson with  85 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/albanyor/data/albanyor_nshft_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/albanyor/data/albanyor_nshft_data_v3.json
Processed 85 features to use
Created GeoJson with  85 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/albanyor/data/albanyor_pshft_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/

Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/alexandriala/data/alexandriala_percent_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/alexandriala/data/alexandriala_percent_data_v3.json
Processed 112 features to use
Created GeoJson with  112 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/alexandriala/data/alexandriala_nshft_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/alexandriala/data/alexandriala_nshft_data_v3.json
Processed 112 features to use
Created GeoJson with  112 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/alexandriala/data/alexandriala_pshft_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/alexandriala/data/alexandriala_pshft_data_v3.json
Processed 112 features to use
Created GeoJson with  112 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/alexandriala/data/alexandriala_growth_v3.js

Processed 204 features to use
Created GeoJson with  204 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/amarillotx/data/amarillotx_nshft_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/amarillotx/data/amarillotx_nshft_data_v3.json
Processed 204 features to use
Created GeoJson with  204 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/amarillotx/data/amarillotx_pshft_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/amarillotx/data/amarillotx_pshft_data_v3.json
Processed 204 features to use
Created GeoJson with  204 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/amarillotx/data/amarillotx_growth_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/amarillotx/data/amarillotx_growth_data_v3.json
Processed 204 features to use
Created GeoJson with  204 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/amar

Processed 251 features to use
Created GeoJson with  251 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/annarbormi/data/annarbormi_pshft_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/annarbormi/data/annarbormi_pshft_data_v3.json
Processed 251 features to use
Created GeoJson with  251 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/annarbormi/data/annarbormi_growth_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/annarbormi/data/annarbormi_growth_data_v3.json
Processed 251 features to use
Created GeoJson with  251 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/annarbormi/data/annarbormi_pshftna_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/annarbormi/data/annarbormi_pshftna_data_v3.json
Processed 251 features to use
Created GeoJson with  251 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/

...

Processed 3059 features to use
Created GeoJson with  3059 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/southcarolina/data/southcarolina_growth_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/southcarolina/data/southcarolina_growth_data_v3.json
Processed 3059 features to use
Created GeoJson with  3059 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/southcarolina/data/southcarolina_pshftna_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/southcarolina/data/southcarolina_pshftna_data_v3.json
Processed 3059 features to use
Created GeoJson with  3059 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/southcarolina/data/southcarolina_growthna_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/southcarolina/data/southcarolina_growthna_data_v3.json
Processed 3059 features to use
Created GeoJson with  3059 features
Wrote geojson d

Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/texas/data/texas_growth_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/texas/data/texas_growth_data_v3.json
Processed 15811 features to use
Created GeoJson with  15811 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/texas/data/texas_pshftna_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/texas/data/texas_pshftna_data_v3.json
Processed 15811 features to use
Created GeoJson with  15811 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/texas/data/texas_growthna_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/texas/data/texas_growthna_data_v3.json
Processed 15811 features to use
Created GeoJson with  15811 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/texas/data/texas_labels_v3.json
Wrote geojson labels data to  /Users/alanmccann/Dropbox/bain/mtsds-data/tex

Processed 5327 features to use
Created GeoJson with  5327 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/virginia/data/virginia_growthna_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/virginia/data/virginia_growthna_data_v3.json
Processed 5327 features to use
Created GeoJson with  5327 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/virginia/data/virginia_labels_v3.json
Wrote geojson labels data to  /Users/alanmccann/Dropbox/bain/mtsds-data/virginia_labels_v3.json

Market Data: 429
Loaded 374 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/Washington_Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/washington_bg_settings_data_v3.json
Loaded 4783 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/washington_Mapping_BG_Data.csv
Processed 4783 features t

Processed 4489 features to use
Created GeoJson with  4489 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/wisconsin/data/wisconsin_growthna_v3.json
Wrote centroid json data to  /Users/alanmccann/Dropbox/bain/mtsds-data/wisconsin/data/wisconsin_growthna_data_v3.json
Processed 4489 features to use
Created GeoJson with  4489 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/wisconsin/data/wisconsin_labels_v3.json
Wrote geojson labels data to  /Users/alanmccann/Dropbox/bain/mtsds-data/wisconsin_labels_v3.json

Market Data: 432
Loaded 374 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/Wyoming_Mapping_Settings_Transposed File_BG.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/wyoming_bg_settings_data_v3.json
Loaded 410 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/wyoming_Mapping_BG_Data.csv
Processed 410 features to us

/var/folders/dh/fn0cs6ln6sq0722wd5lcgdcm0000gn/T/ipykernel_95264/1269797648.py:28: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')


In [10]:
def upload_to_mapbox(source_data_base_path,published_data_base_path):
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows: #[0:10]:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            os.system(f'cd {published_data_base_path}; tilesets upload-source stobieb {market_file_prefix}_bg_data-src_v{MAP_VERSION} {market_file_prefix}_bg_data_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION} --recipe {market_file_prefix}-bg-recipe_v{MAP_VERSION}.json --name "{market_file_prefix}_bg_data"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION}')
            os.system(f'cd {published_data_base_path};  tilesets upload-source stobieb {market_file_prefix}_bg_labels-src {market_file_prefix}_bg_labels_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_labels --recipe {market_file_prefix}-labels-recipe.json --name "{market_file_prefix}_bg_labels_v{MAP_VERSION}"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_labels')

In [11]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}

import subprocess
# tilesets job stobieb.dallasfortworth_bg_data cl9kgdofo000209l52zyy3mdi
process = subprocess.Popen(['tilesets', 'job', 'stobieb.dallasfortworth_bg_data', 'cl9kgdofo000209l52zyy3mdi'],
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)
stdout, stderr = process.communicate()
output = stdout.decode('utf-8').split('\n')
error = stderr.decode('utf-8')
result = json.loads(output[0])



In [12]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
import subprocess

cwd = os.getcwd()
os.chdir(published_data_base_path)

market_file_prefix = 'austin'
stats_type = 'Count'

def upload_source(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_{stats_type.lower()}_src_v{MAP_VERSION}',
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json',
        '--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)

def upload_label_source(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_labels_src_v{MAP_VERSION}',
        f'{market_file_prefix}_labels_v{MAP_VERSION}.json',
        #'--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)
    
# upload_source(market_file_prefix, stats_type, published_data_base_path)

In [13]:
def update_recipe(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def upload_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        '--recipe', 
        f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

def upload_label_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        '--recipe', f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

In [14]:
import time
def publish_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}'
    ]

    print(' '.join(publish_tileset_command))
    print(datetime.datetime.now())
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(15)
    print('finished')
    print(datetime.datetime.now())


In [22]:
import csv
import datetime
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
# STATS_TYPES = ['count','percent','nshft','labels','growth', 'pshft','pshftna','growthna']
STATS_TYPES = ['growth']
# STATS_TYPES = ['nshft']
# Ohio market_rows[416:417]
# Cleveland market_rows[73:74]
count = 367
for market_data in market_rows[368:]: #[416:417]:
    market_file_prefix = market_data['MarketName'].lower()
    if True: #market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
        count += 1
        print('count',count)
        print('creating tilesets for:', market_file_prefix)
        print(datetime.datetime.now())
        for stats_type in STATS_TYPES:
            upload_source(market_file_prefix, stats_type, published_data_base_path)
            upload_tileset(market_file_prefix, stats_type, published_data_base_path)
            update_recipe(market_file_prefix, stats_type, published_data_base_path)
            publish_tileset(market_file_prefix, stats_type, published_data_base_path)
        
        

count 368
creating tilesets for: wheelingwv
2023-07-21 10:14:19.979803
tilesets upload-source stobieb wheelingwv_growth_src_v3 wheelingwv_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/wheelingwv_growth_src_v3", "files": 1, "source_size": 2050566, "file_size": 2050566}', '']
tilesets create stobieb.wheelingwv_growth_v3 --recipe wheelingwv_growth_recipe_v3.json --name wheelingwv_growth_v3
['{"message": "stobieb.wheelingwv_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.wheelingwv_growth_v3 wheelingwv_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.wheelingwv_growth_v3
2023-07-21 10:14:26.264041
['{"message": "Processing stobieb.wheelingwv_growth_v3", "jobId": "clkcnx8k2002s08msepwl40qf"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.wheelingwv_growth_v3 or run tilesets job stobieb.wheelingwv_growth_v3 clkcnx8k2002s08msepwl40qf to view the status of your tileset.

{'message': 'P

['upload progress', '{"id": "mapbox://tileset-source/stobieb/winstonnc_growth_src_v3", "files": 1, "source_size": 4350260, "file_size": 4350260}', '']
tilesets create stobieb.winstonnc_growth_v3 --recipe winstonnc_growth_recipe_v3.json --name winstonnc_growth_v3
['{"message": "stobieb.winstonnc_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.winstonnc_growth_v3 winstonnc_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.winstonnc_growth_v3
2023-07-21 10:32:31.624266
['{"message": "Processing stobieb.winstonnc_growth_v3", "jobId": "clkcokhvx000r08jo9l3e7cmf"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.winstonnc_growth_v3 or run tilesets job stobieb.winstonnc_growth_v3 clkcokhvx000r08jo9l3e7cmf to view the status of your tileset.

{'message': 'Processing stobieb.winstonnc_growth_v3', 'jobId': 'clkcokhvx000r08jo9l3e7cmf'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: 

['{"message": "stobieb.yumaaz_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.yumaaz_growth_v3 yumaaz_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.yumaaz_growth_v3
2023-07-21 10:50:44.963451
['{"message": "Processing stobieb.yumaaz_growth_v3", "jobId": "clkcp7xr5000508ml75uq9qc5"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.yumaaz_growth_v3 or run tilesets job stobieb.yumaaz_growth_v3 clkcp7xr5000508ml75uq9qc5 to view the status of your tileset.

{'message': 'Processing stobieb.yumaaz_growth_v3', 'jobId': 'clkcp7xr5000508ml75uq9qc5'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 10:53:32.699690
count 381
creating tilesets for: alabama
2023-07-21 10:53:32.699863
tilesets upload-source stobieb alabama_growth_src_v3 alabama_growth_v

[''] Updated recipe.

tilesets publish stobieb.colorado_growth_v3
2023-07-21 11:15:06.229889
['{"message": "Processing stobieb.colorado_growth_v3", "jobId": "clkcq3968004908mbc902a8ni"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.colorado_growth_v3 or run tilesets job stobieb.colorado_growth_v3 clkcq3968004908mbc902a8ni to view the status of your tileset.

{'message': 'Processing stobieb.colorado_growth_v3', 'jobId': 'clkcq3968004908mbc902a8ni'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 11:18:26.321219
count 387
creating tilesets for: connecticut
2023-07-21 11:18:26.321352
tilesets upload-source stobieb connecticut_growth_src_v3 connecticut_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb

['{"message": "Processing stobieb.hawaii_growth_v3", "jobId": "clkcqsobe002808k15b70084x"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.hawaii_growth_v3 or run tilesets job stobieb.hawaii_growth_v3 clkcqsobe002808k15b70084x to view the status of your tileset.

{'message': 'Processing stobieb.hawaii_growth_v3', 'jobId': 'clkcqsobe002808k15b70084x'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 11:37:54.718179
count 393
creating tilesets for: idaho
2023-07-21 11:37:54.718317
tilesets upload-source stobieb idaho_growth_src_v3 idaho_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/idaho_growth_src_v3", "files": 1, "source_size": 18007837, "file_size": 18007837}', '']
tilesets create stobieb.idaho_growth_v3 --recipe id

...

processing: sleeping
finished
2023-07-21 12:19:03.771306
count 405
creating tilesets for: mississippi
2023-07-21 12:19:03.771425
tilesets upload-source stobieb mississippi_growth_src_v3 mississippi_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/mississippi_growth_src_v3", "files": 1, "source_size": 39734300, "file_size": 39734300}', '']
tilesets create stobieb.mississippi_growth_v3 --recipe mississippi_growth_recipe_v3.json --name mississippi_growth_v3
['{"message": "stobieb.mississippi_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.mississippi_growth_v3 mississippi_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.mississippi_growth_v3
2023-07-21 12:19:20.729238
['{"message": "Processing stobieb.mississippi_growth_v3", "jobId": "clkcsdvng001g08js7912a3xg"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.mississippi_growth_v3 or run tilesets job stobieb.mississippi_growth_v3 clkc

['upload progress', '{"id": "mapbox://tileset-source/stobieb/newjersey_growth_src_v3", "files": 1, "source_size": 26900188, "file_size": 26900188}', '']
tilesets create stobieb.newjersey_growth_v3 --recipe newjersey_growth_recipe_v3.json --name newjersey_growth_v3
['{"message": "stobieb.newjersey_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.newjersey_growth_v3 newjersey_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.newjersey_growth_v3
2023-07-21 12:40:34.206408
['{"message": "Processing stobieb.newjersey_growth_v3", "jobId": "clkct55z1000708mqe9hzfutf"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.newjersey_growth_v3 or run tilesets job stobieb.newjersey_growth_v3 clkct55z1000708mqe9hzfutf to view the status of your tileset.

{'message': 'Processing stobieb.newjersey_growth_v3', 'jobId': 'clkct55z1000708mqe9hzfutf'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing

['{"message": "stobieb.oklahoma_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.oklahoma_growth_v3 oklahoma_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.oklahoma_growth_v3
2023-07-21 13:02:05.928612
['{"message": "Processing stobieb.oklahoma_growth_v3", "jobId": "clkctwuls002g08l0a570cojk"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.oklahoma_growth_v3 or run tilesets job stobieb.oklahoma_growth_v3 clkctwuls002g08l0a570cojk to view the status of your tileset.

{'message': 'Processing stobieb.oklahoma_growth_v3', 'jobId': 'clkctwuls002g08l0a570cojk'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 13:05:09.355974
count 418
creating tilesets for: oregon
2023-07-21 13:05:09.356113
tilesets upload-source stobieb or

[''] Updated recipe.

tilesets publish stobieb.tennessee_growth_v3
2023-07-21 13:22:18.900244
['{"message": "Processing stobieb.tennessee_growth_v3", "jobId": "clkcumusu001e08l30wzc7vha"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.tennessee_growth_v3 or run tilesets job stobieb.tennessee_growth_v3 clkcumusu001e08l30wzc7vha to view the status of your tileset.

{'message': 'Processing stobieb.tennessee_growth_v3', 'jobId': 'clkcumusu001e08l30wzc7vha'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-07-21 13:25:23.150764
count 424
creating tilesets for: texas
2023-07-21 13:25:23.150920
tilesets upload-source stobieb texas_growth_src_v3 texas_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/texas_growth_src_v3", "files": 1,

queued: sleeping
finished
2023-07-21 13:44:10.457674
count 430
creating tilesets for: wisconsin
2023-07-21 13:44:10.457792
tilesets upload-source stobieb wisconsin_growth_src_v3 wisconsin_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/wisconsin_growth_src_v3", "files": 1, "source_size": 37520014, "file_size": 37520014}', '']
tilesets create stobieb.wisconsin_growth_v3 --recipe wisconsin_growth_recipe_v3.json --name wisconsin_growth_v3
['{"message": "stobieb.wisconsin_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.wisconsin_growth_v3 wisconsin_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.wisconsin_growth_v3
2023-07-21 13:44:27.097031
['{"message": "Processing stobieb.wisconsin_growth_v3", "jobId": "clkcvfbg1000108mi2nmvbvt3"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.wisconsin_growth_v3 or run tilesets job stobieb.wisconsin_growth_v3 clkcvfbg1000108mi2nmvbvt3 to view th

In [26]:
import csv
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)

In [54]:
market_rows[416:417]

[{'MarketName': 'Ohio',
  'Market_Display_Name': 'Ohio',
  'StateFips': '',
  'CenterLat': '40.2862',
  'CenterLong': '-82.7937',
  'GeoTypeGroup': '2 - States',
  'DefaultZoomLevel': '5',
  'InterMarketMetro': '0'}]

In [ ]:
'/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/AbileneTX__Mapping_Settings_Transposed File_BG.csv'

'/Users/alanmccann/Dropbox/bain/bain-uploads/US_Geospatial_2023_Run5 brian stobie/AbileneTX_Mapping_Settings_Transposed File_BG.csv'
